<a href="https://colab.research.google.com/github/morutkin/MLProject/blob/master/kuzushiji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Momo:
- tSNE
- KNN 


Seo:
- pre process 
 * char crops for model input 
 * image crop with multiple chars 
    * should have bounding boxes around chars 
    * figure out most ideal crop sizes 

* Two inputs to the models
 * one character crops 
 * image crops with multiple characters (each one would 
 have a bounding box around it 
* for the segmentation into characters, use Faster-RCNN model with resnet152 backbone trained with torchvision detection. Only one class 
  * resnet152 backbone worked a bit better than default resnet50 
  * pipeline was modified to accept empty crops, (crops without ground truth objects) to reduce amount of false positives,
  * it was trained on 512x384 crops
* improvement potentials 
  * mmdetection - can be used for better object detection 
  * blending predictions from different folds for submission
  * SGD with momentum 
  * resnext101_32x8d_wsl with psudo labeling is the best op 80% most confident predictions from the blend were used, having accuracy >99% according to validation. Next, two kinds of models were trained (all based on resnext101_32x8d_wsl):
      * models from previous step fine-tuned for 5 epochs (compared to 50 epochs for training from scratch) with starting learning rate 10x smaller than initial learning rate.
models trained from scratch with default settings.
  * Overall, many improvement are possible here, from just using bigger models and freezing less layers, to more work on training schedule, augmentations, etc.
  * XGBoost and LighGBM models were trained across all folds, and then blended.

Important stuff to note:
* we have to create our own bounding boxes around the test set characters, the 2nd place winners suggest that we use resnet(which predicts objects) and do cross validation to get the omost accurate box classifications 
* after this happens, then we would classify the characters

KNN 
* https://github.com/lopuhin/kaggle-kuzushiji-2019/blob/master/kuzushiji/classify/knn.py

Websites:
https://www.kaggle.com/c/kuzushiji-recognition/discussion/112712#latest-675509


# Kuzushiji Translation






### Team Members: Momo Rutkin, Seo Choe

## Description:
Japan is a country with thousands of years of culture with written records dating back to the 8th century.  Many of the books and letters written unveils a lot about Japanese portrayal of various aspects of life, including stories and anecdotes known throughout their people. However, a lot of these written materials are unrecognizable to modern Japanese people due to the change in language over the years. One of these big transformations was from Kuzushiji to modern Japanese. 
<br> <br> Kuzushiji is a Japanese cursive writing style that has stopped being used 150 years ago when the Japanese education system had a reformation. Although some of the characters reflect similar characters used in the modern Japanese language, many of the characters are still hard to make meaning of. Although most of Japans historical books are written in this form, only 0.01% of Japanese people living today can read them fluently. Due to the lack of human resources, there has been huge efforts to use Machine Learning to transcribe these ancient texts into modern Japanese characters. However, there are many challenges that make character recognition efforts challenging for Machine Learning purposes. 





## Some of these challenges include:
 

*   A large number of character types with a majority of the characters used in frequently, resulting in an unbalanced character representation within the dataset 
*   While in modern Japanese, theres only a single way of writing each character, in Kuzushiji, there were many different ways of writing the same character. This phenomenon is called Hentaigana
*   Some unique characters look very similar so it’s hard to distinguish between the two 
*   Connectedness and overlap between the characters since this is a cursive script  
*   Various layouts where Kuzushiji is written 

## Goal: 
To take the data from ROIS-DS Center For Humanities and build a model that can locate and classify Kuzushiji-characters on images.



## Data:

### Train.csv: 
 * <b>Image_id</b>: The filename used to uniquely identify each image
 * <b>labels</b>: A string containing all labels for the given image. This can be used to draw a bounding box around each character on the image. The string contains each Unicode character, X-coordinate, Y-coordinate, With and Height. The information are space seperated.
 
### Unicode_translations.csv:
 * Mappings of the unicode ID to the modern Japanese character

### train_images.zip:
* Training images
* All the characters in these images are classified in train.csv

### test_images.zip:
* Testing images



## EDA Steps:
* First, we would like to do EDA to get a better feel for the most frequent characters within the training set. We would do this using T-SNE, which can visually group together common characters




* We also want to take the competitions given data visualizer seen in: https://www.kaggle.com/anokas/kuzushiji-visualisation to see the labels associated with the characters in the images for the training data as well. This will give us a better understanding of the similarities and differences between the data and the associated labels. 



## Preprocessing Steps:
* Due to the high imablance in the dataset in terms of representation of characters, we would want to think of ensemble classifiers that can help balance out the different classifications via a balanced bagging classifier for resampling before training classifiers.

* 

* PCA




## ML Steps:
### Supervised:
* KNN
* Naive Bayes 


### Unsupervised:
* CNN - XGBoost. loss function on CNN results. feed output of CNN to XGboost  
* CNN - SVM 

### What a sample Submission Looks Like:

### How are submissions evaluated?: F1-Score
* Talk about what the output would look like:
* https://www.kaggle.com/anokas/kuzushiji-modified-f-1-score -- talk about what their F1 score metric does 




# Ways to improve Accuracy 
* use the dropout method when training neural nets - reduces overfitting 
* train parameters using kfolds 
* regularization penality for loss funtion for NN


### References:
https://www.kaggle.com/christianwallenwein/visualization-useful-functions-small-eda

https://towardsdatascience.com/deep-learning-for-classical-japanese-literature-48ae04c17dfd


# Data Imports

For each time you open Collab, make sure to run all of the code in order. 

In [1]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)


Download 100%.


In [0]:
pip install kaggle

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [11]:
!kaggle competitions list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
!kaggle competitions files -c kuzushiji-recognition

name                      size  creationDate         
-----------------------  -----  -------------------  
unicode_translation.csv   51KB  2019-07-25 14:32:40  
sample_submission.csv    146KB  2019-07-25 14:32:47  
train.csv                 15MB  2019-07-25 14:32:48  
test_images.zip            4GB  2019-07-25 14:32:48  
train_images.zip           3GB  2019-07-25 14:32:48  


In [14]:
!kaggle competitions download  -c kuzushiji-recognition -p /content/kaggle

  0% 0.00/51.5k [00:00<?, ?B/s]
100% 51.5k/51.5k [00:00<00:00, 45.4MB/s]
  0% 0.00/146k [00:00<?, ?B/s]
100% 146k/146k [00:00<00:00, 45.4MB/s]
 88% 5.00M/5.67M [00:00<00:00, 27.2MB/s]
100% 5.67M/5.67M [00:00<00:00, 27.8MB/s]
100% 3.58G/3.58G [00:51<00:00, 41.4MB/s]
100% 3.58G/3.58G [00:51<00:00, 74.9MB/s]
100% 3.03G/3.03G [00:40<00:00, 91.4MB/s]
100% 3.03G/3.03G [00:40<00:00, 81.3MB/s]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Data Pre Processing

The data preprocessing will consist of 
* <b>image cropping + resizing for uniform size</b>:
  * for single characters
  * for multiple characters in a crop 

<b> Possible additional preprocessing </b>
* turning the color of the characters to a uniform black 
* For NN
 * rescale continuous features 
 * diagnal covariance matrix 
 * whiten data 


In [0]:
#for Noto font for kanji 
!wget -q --show-progress https://noto-website-2.storage.googleapis.com/pkgs/NotoSansCJKjp-hinted.zip
!unzip -p NotoSansCJKjp-hinted.zip NotoSansCJKjp-Regular.otf > NotoSansCJKjp-Regular.otf
!rm NotoSansCJKjp-hinted.zip
import matplotlib.font_manager as font_manager
path = './NotoSansCJKjp-Regular.otf'
prop = font_manager.FontProperties(fname=path)

NotoSansCJKjp-hinte 100%[===================>] 115.49M  47.8MB/s    in 2.4s    


In [0]:
#for mmdetection 
pip install mmcv

In [0]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh && bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')


--2019-12-03 19:42:59--  https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 651745206 (622M) [application/x-sh]
Saving to: ‘Anaconda3-5.2.0-Linux-x86_64.sh’

Anaconda3-5.2.0-Lin 100%[===================>] 621.55M   183MB/s    in 3.4s    

2019-12-03 19:43:02 (185 MB/s) - ‘Anaconda3-5.2.0-Linux-x86_64.sh’ saved [651745206/651745206]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: blas-1.0-mkl ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-8 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libgfortran-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: bzip2-1.0.6

In [0]:
conda create -n open-mmlab python=3.7 -y

SyntaxError: ignored

In [0]:
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local

In [0]:
# All Imports 

import re
import pandas as pd
import numpy as np
from pathlib import Path
import mmcv
from PIL import Image
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensor
import torch.utils.data
from mmdet.core.bbox import bbox_overlaps

In [0]:
#unzip pictures in train 

import zipfile

with zipfile.ZipFile('/content/kaggle/train_images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/kaggle/train_images')

with zipfile.ZipFile('/content/kaggle/test_images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/kaggle/test_images')

In [0]:
#All data imports 

train_set = pd.read_csv('/content/kaggle/train.csv.zip', compression='zip', header=0, sep=',', quotechar='"', keep_default_na=False)
#print(train_set.head())
print(train_set["labels"])
unimap = pd.read_csv('/content/kaggle/unicode_translation.csv')
sample_submiss = pd.read_csv('/content/kaggle/sample_submission.csv')
img_dir = Path('/content/kaggle/train_images')
img_dirTest = Path('/content/kaggle/test_images')
print(os.listdir('/content/kaggle/train_images'))



0       U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...
1       U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...
2       U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...
3       U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...
4       U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...
                              ...                        
3876    U+309D 426 456 30 34 U+306F 584 359 82 45 U+30...
3877    U+30DE 677 327 68 35 U+3078 1424 463 110 43 U+...
3878    U+309D 1582 802 16 34 U+306F 597 2337 72 42 U+...
3879    U+4E00 1050 898 86 21 U+309D 724 1864 27 29 U+...
3880                                                     
Name: labels, Length: 3881, dtype: object
['100249537_00046_2.jpg', 'hnsd009-040.jpg', '200004148_00045_2.jpg', '200021869-00015_2.jpg', 'hnsd002-001.jpg', 'umgy004-007.jpg', 'umgy008-032.jpg', '200021637-00019_1.jpg', '200021660-00043_1.jpg', '200021644_00020_1.jpg', '200015779_00043_1.jpg', 'umgy001-003.jpg', 'umgy011-019.jpg', '100249476_00006_1.jpg', '2000

In [0]:
def cropImage(labels, img_dir):



  labels = labels.split(" ")

  labels[::5] = map(unicodeToInt, labels[::5])
  
  unicode = labels[::5]
  del labels[::5]
  labels = np.array(labels, dtype=np.int16)
  labels = labels.reshape(-1,4)

  labels[:, 2] = np.sum(a=labels[:,[0,2]], axis=1)
  labels]:, 3] = np.sum(a=labels[:,[1,3]], axis=1)

  [Image.fromarray(img[label[1]:label[3], label[0]:label[2]]).save(f"{CHARS}/{unicode[]}_{}-{}.jpg") for ,label in enumerate(labels)]

# Useful Functions

In [0]:
def toPath(string):
    ''' image_id to the path to image '''
    if ".jpg" not in string:
        string = string + ".jpg"
    return string

def toID(string):
    ''' image path to image_id '''
    if string[-4:] ==".jpg":
        string = string[:-4]
    return string



In [0]:
def to_coco(boxes: torch.Tensor) -> torch.Tensor:
    """ Convert from pytorch detection format to COCO format.
    """
    boxes = boxes.clone()
    boxes[:, 2] -= boxes[:, 0]
    boxes[:, 3] -= boxes[:, 1]
    return boxes

def from_coco(boxes: torch.Tensor) -> torch.Tensor:
    """ Convert from CODO to pytorch detection format.
    """
    boxes = boxes.clone()
    boxes[:, 2] += boxes[:, 0]
    boxes[:, 3] += boxes[:, 1]
    return boxes

def scale_boxes(
        boxes: torch.Tensor, w_scale: float, h_scale: float) -> torch.Tensor:
    return torch.stack([
        boxes[:, 0] * w_scale,
        boxes[:, 1] * h_scale,
        boxes[:, 2] * w_scale,
        boxes[:, 3] * h_scale,
        ]).t()

def get_target_boxes_labels(item):
    if item.labels:
        labels = np.array(item.labels.split(' ')).reshape(-1, 5)
    else:
        labels = np.zeros((0, 5))
    boxes = labels[:, 1:].astype(np.float)
    labels = labels[:, 0]
    return boxes, labels

In [0]:
def get_transform(train):
    train_initial_size = 2048
    crop_min_max_height = (400, 533)
    crop_width = 512
    crop_height = 384
    if train:
        transforms = [
            A.LongestMaxSize(max_size=train_initial_size),
            A.RandomSizedCrop(
                min_max_height=crop_min_max_height,
                width=crop_width,
                height=crop_height,
                w2h_ratio=crop_width / crop_height,
            ),
            A.HueSaturationValue(
                hue_shift_limit=7,
                sat_shift_limit=10,
                val_shift_limit=10,
            ),
            A.RandomBrightnessContrast(),
            A.RandomGamma(),
        ]
    else:
        test_size = int(train_initial_size *
                        crop_height / np.mean(crop_min_max_height))
        print(f'Test image max size {test_size} px')
        transforms = [
            A.LongestMaxSize(max_size=test_size),
        ]
    transforms.extend([
        ToTensor(),
    ])
    return A.Compose(
        transforms,
        bbox_params={
            'format': 'coco',
            'min_area': 0,
            'min_visibility': 0.5,
            'label_fields': ['labels'],
        },
    )



In [0]:
# In the training data, we get an entire string with all the characters in it and that needs to be splitted
# new list, every element is one char + all the information needed to create the bounding box
def splitEachChar(string):
    string = str(string)
    string = (re.findall(r"(?:\S*\s){5}", string))
    return [line[:-1]for line in string]

# new list, split everything by a blank
def splitEachInformation(string):
    string = str(string)
    string = string.split(" ")
    return string
        
    
    
print(splitEachChar(train_set.labels[0])[:2])
print(splitEachInformation(train_set.labels[0])[:5])

['U+306F 1231 3465 133 53', 'U+304C 275 1652 84 69']
['U+306F', '1231', '3465', '133', '53']


In [0]:
#function to get all unicode chars from a string with regex
def get_unicodes(string):
    string = str(string)
    return re.findall(r'U[+][\S]*', string)

#returns the image size given the image_id or path to the image
def getImageSize(image):
    path = toPath(image)
    width, height = Image.open(img_dir/path).size
    return [width, height]

#getImageSize(test_img_id)

In [0]:
#counts of all the times a unicode has come up in translation. Can be used for EDA

#a mapping of unicode to the actual char 
unicode_map = {codepoint: char for codepoint, char in unimap.values}
#list form of all the unicodes in the transation, no repeats 
unicode_list = list(unicode_map)
print(unicode_map)

#unicode to the class ID
unicode2class = dict(zip(unimap['Unicode'], unimap.index.values))
print(unicode2class)

#turns the unicode into the actual character
def unicodeToCharacter(unicode):
    return unicode_map[unicode]

# unicode to int conversion and the other way around
# unique identifier for every unicode character
def unicodeToInt(unicode):
    return unicode_list.index(unicode)

def intToUnicode(integer):
    return unicode_list[integer]

test_unicode = unicode_list[10]
print(intToUnicode(57))

#iterate through bbox parameters 
def iter_bboxes(labels):
    if not labels:
        return
    labels = labels.split()
    n = len(labels)
    assert n % 5 == 0
    for i in range(0, n, 5):
        ch, x, y, w, h = labels[i:i + 5]
        yield int(x), int(y), int(w), int(h), ch

## Creates Bounding Boxes
* bboxes for train is from the labels 
* gives backbones for the test 

In [0]:
 def prepare_train():  
    # Add images to COCO
    images = []
    for img_id, row in tqdm(train_set.iterrows()):
        filename = row['image_id'] + '.jpg'
        img = Image.open(img_dir / filename)
        image = {
            'filename': filename,
            'width': img.width,
            'height': img.height,
        }
        bboxes = []
        labels = []
        for x, y, w, h, ch in iter_bboxes(row['labels']):
            bboxes.append([x, y, x + w, y + h])
            # labels.append(1)
            labels.append(unicode2class[ch] + 1)
        image['ann'] = {
            'bboxes': np.array(bboxes).astype(np.float32).reshape(-1, 4),
            'labels': np.array(labels).astype(np.int64).reshape(-1),
            'bboxes_ignore': np.array([], dtype=np.float32).reshape(-1, 4),
            'labels_ignore': np.array([], dtype=np.int64).reshape(-1, )
        }
        images.append(image)
    #print(images[1])
    import random
    random.shuffle(images)
    #os.mkdir('/content/kaggle/data')
    open('/content/kaggle/data/dval.pkl', 'a').close()
    open('/content/kaggle/data/dtrain.pkl', 'a').close()
    open('/content/kaggle/data/dtrainval.pkl', 'a').close()
    open('/content/kaggle/data/dtest.pkl', 'a').close()
    print(os.listdir('/content/kaggle/data'))
    mmcv.dump([im for im in images if im['filename'].startswith('umgy')], '/content/kaggle/data/dval.pkl')
    mmcv.dump([im for im in images if not im['filename'].startswith('umgy')], '/content/kaggle/data/dtrain.pkl')
    mmcv.dump(images, '/content/kaggle/data/dtrainval.pkl')



In [0]:
def prepare_test():
    

    images = []
    for img_id, row in tqdm(sample_submiss.iterrows()):
        filename = row['image_id'] + '.jpg'
        img = Image.open(img_dirTest / filename)
        images.append({
            'filename': filename,
            'width': img.width,
            'height': img.height,
            'ann': {
                'bboxes': np.array([], dtype=np.float32).reshape(-1, 4),
                'labels': np.array([], dtype=np.int64).reshape(-1, ),
                'bboxes_ignore': np.array([], dtype=np.float32).reshape(-1, 4),
                'labels_ignore': np.array([], dtype=np.int64).reshape(-1, )
            }
        })
    mmcv.dump(images, '/content/kaggle/data/dtest.pkl')

In [0]:
prepare_train()
prepare_test()
print(len([im for im in images if im['filename'].startswith('umgy')]))
print(len(images))

496
3881


## Another bbox image display from the visualization notebook

In [0]:
def isUnicode(string):
    ''' check whether the passed string is a unicode or not '''
    string = string.strip()
    if re.match("^U\+\w{4,5}$", string):
        return(True)
    else:
        return(False)
    

In [0]:
def displayImage(filepath=None, directory=None, image_id=None):
    """
    display one image with matplotlib, the actual image
    
    Parameters:
    - either specify the entire filepath or (the direcory and the image_id)
    
    Returns:
    - matplotlib.pyplot figure of the image
    """
    
    if filepath == None:
        if (directory == None) and (image_id==None):
            print("path to file not specified")
            return None
        else:
            filepath=directory/toPath(image_id)
    
    plt.figure(figsize=(15,15))
    this_img = Image.open(filepath)
    plt.imshow(this_img)
    return plt

#displayImage(directory=TRAIN, image_id=test_img_id)

In [0]:
# take the image_id return an image with bounding boxes around each character
# image_id is the filename without the file extension (in this case .jpg)

# get all the characters and the position of the bounding boxes for an image
def getLabels(image_id):
    allLabels = train_set.loc[train_df["image_id"]==image_id].labels[0]
    allLabels = np.array(allLabels.split(" ")).reshape(-1, 5)
    return allLabels

def drawBoxAndText(ax, label):
    codepoint, x, y, w, h = label
    x, y, w, h = int(x), int(y), int(w), int(h)
    rect = Rectangle((x, y), w, h, linewidth=1, edgecolor="r", facecolor="none")
    ax.add_patch(rect)
    ax.text(x+w+25, y+(h/2)+20, unicodeToCharacter(codepoint),
            fontproperties=prop,
            color="r",
           size=16)
    return ax

def displayTrainingData(image_id):
    labels = getLabels(image_id)
    plt = displayImage(directory=TRAIN, image_id=image_id)
    ax = plt.gca()

    for label in labels:
        ax = drawBoxAndText(ax, label)
        
    
#displayTrainingData(test_img_id)

In [0]:
def filenameToUnicodeInt(string):
    '''
    filename to integer representing a unicode
    '''
    unicode = string.split("_")[0]
    unicodeInteger = unicodeToInt(unicode)
    return unicodeInteger

# EDA


In [0]:
#concatenate all labels to one string
all_labels = train_set.labels.str.cat(sep=" ")

# get all unicodes in that string
all_unicodes = get_unicodes(all_labels)

# get the number of unique values from all unicodes
len(set(all_unicodes))

#Only 4212 of the 4787 characters are in the training set. Thus some characters definitely can't be predicted.

In [0]:
#drop the images with no labels 
train_set.isnull().sum()


In [0]:

train_set.dropna(inplace=True)
train_set.reset_index(inplace=True, drop=True)
print ("TRAIN: ", train_set.shape)

# Character Segmentation - Input Type 1
